#  Homework Assignment 5
## Nana Wang (ID: 999480980)

## Excercise 4 [Implementation of augmented Lagrangian method]

### Functions

In [1]:
#Modified Newton method and quasi-Newton method （bfgs)
function newtmin(obj,x0,method;eps=1,maxIts=100,optTol=1e-8,r=0.9,c1=1e-4,c2=0.9)
    # Minimize a function f using Netown's method.
    #obj: a function that evaluates the objective values, gradient and Hessian at a point x, i.e., (f,g,H)=obj(x)
    #x0: starting point
    #maxIts: maximum number of iterations
    #optTol: optimality toleracne based on ||grad(x)||<=optTol*||grad(x0)||
    if method=="newton"
        g=obj(x0)[2]
        H=obj(x0)[3]
        its=0;g0=g;x=x0
        #G=Float64[]
        while norm(g)>optTol && its<maxIts
            #push!(G,norm(g))
            D,V=eig(H)
            H=V*Diagonal(map(x->1/max(abs(x),eps),D))*V'
            d=-H*g
            x=x+btls(obj,x,d,g,r,c1)*d
            its=its+1
            g=obj(x)[2]
            H=obj(x)[3]
        end
    elseif method=="bfgs"
        f0,g0,H0=obj(x0)
        D,V=eig(H0)
        H0=V*Diagonal(map(x->1/max(x,1),D))*V'
        its=0;ng=norm(g0);x=x0
        #G=Float64[]
        while norm(g0)>optTol && its<maxIts
            #push!(G,norm(g0))
            d=-H0*g0
            x=x0+bisection_wolfe(obj,x0,d,c1,c2,r)*d
            g=obj(x)[2]
            s=x-x0; y=g-g0
            rho=1/dot(y,s)
            H=(eye(H0)-rho*s*y')*H0*(eye(H0)-rho*y*s')+rho*s*s'
            its=its+1
            x0=x;g0=g;H0=H
        end
    else 
        println("Please enter netwon or bfgs!")
    end
    return x,its
end

function btls(obj,x,d,g,r,c1;a=1,maxIts=100)
    for i=1:maxIts
        a=a*r
        if obj(x+a*d)[1]<=obj(x)[1]+a*c1*dot(g,d)
            break
        end
     end
    return a
end

function bisection_wolfe(obj,x0,d,c1,c2,r; maxIts=25)
    a=0;t=1;b=Inf
    f0,g0=obj(x0)
    for i=1:maxIts
        f,g,H=obj(x0+t*d)
        if f>f0+c1*dot(g0,d)
            b=t; t=a*(1-r)+b*r
        elseif dot(g,d)<c2*dot(g0,d)
            a=t;
            if isinf(b)
                t=2*a
            else
                t=a*(1-r)+b*r
            end
        else
           break;
        end
    end
        return t
end

bisection_wolfe (generic function with 1 method)

In [2]:
import ForwardDiff
function AugLag(f,c,x0,y0,method;ρ=1, rho=10,optTol=1e-8,r=0.9,maxIts=1000)
    k=0;xk=x0;yk=y0;rk=ρ
    g0=ForwardDiff.gradient(f)
    J=ForwardDiff.jacobian(c)
    Jk=J(xk)'
    for i=1:maxIts
        #println("Step ",k,": ", "xk=",xk," rk=",rk,"; |c(xk)|=", norm(c(xk))," , |g-J'y|=", norm(g(xk)-Jk*yk))
        #println("   norm(c(xk))=", norm(c(xk)), " and norm(g-Jy)=",norm(g(xk)-Jk*yk))
        if norm(c(xk))<=optTol && norm(g0(xk)-Jk*yk)<=optTol 
            break;
        end
        function objL(y)
            l(x)=f(x)-dot(yk,c(x))+dot(c(x),c(x))*rk/2
            g=ForwardDiff.gradient(l)
            H=ForwardDiff.hessian(l)
            return (l(y),g(y),H(y))
        end
        xk=newtmin(objL,xk,method,r=r)[1]
        if norm(c(xk)) <= 1/k
            yk=yk-rk*c(xk)
        else
            rk=10*rk
        end
        Jk=J(xk)'
        k=k+1
    end
    if k==maxIts
        println("  Not converge! |c(xk)|=",norm(c(xk))," and |g-J'y|=", norm(g(xk)-Jk*yk))
    end
    return xk, f(xk), k
end

AugLag (generic function with 1 method)

### Problem 6-9 of the Hock-Schittkowski Collection

In [3]:
function testpb(i)
    if i==6
        f(x:: Vector)=(1-x[1])^2
        c(x:: Vector)=[10*(x[2]-x[1]^2)]
        x0=[-1.2,1]
        m=1
    elseif i==7
        f(x:: Vector)=log(1+x[1]*x[1])-x[2]
        c(x:: Vector)=[(1+x[1]*x[1])^2+x[2]*x[2]-4]
        x0=[2.0,2.0]
        m=1
    elseif i==8
        f(x:: Vector)=1
        c(x:: Vector)=[x[1]^2+x[2]^2-25, x[1]*x[2]-9]
        x0=[2.0,1.0]
        m=2
    elseif i==9
        f(x::Vector)=sin(pi*x[1]/12)*cos(pi*x[2]/16)
        c(x::Vector )=[4*x[1]-3*x[2]]
        x0=[0.0,0.0]
        m=1
    else
        println("Please enter 6, 7, 8 or 9!")
    end
    return f,c,x0,m
end

testpb (generic function with 1 method)

### Results

In [4]:
println("Newton's method: ")
srand(1)
for i=6:9
    f,c,x0,m=testpb(i)
    y0=randn(m)
    println("For problem ",i,": ")
    xk,fxk,k=AugLag(f,c,x0,y0,"newton")
    println("  xk=",xk,", f(xk)=", fxk," and its=",k)
end

Newton's method: 
For problem 6: 
  xk=[1.0000000090298693,1.0000000181920625], f(xk)=8.153853951619753e-17 and its=2
For problem 7: 
  xk=[1.009968745605941e-40,1.7320508084964588], f(xk)=-1.7320508084964588 and its=7
For problem 8: 
  xk=[4.601594917663884,1.9558436065355926], f(xk)=1 and its=2
For problem 9: 
  xk=[-2.999999890072189,-3.9999998536882555], f(xk)=-0.49999999997460437 and its=5


In [5]:
println("Quasi-Newton method (BFGS):")
srand(1)
for i=6:9
    f,c,x0,m=testpb(i)
    y0=randn(m)
    println("For problem ",i,": ")
    xk,fxk,k=AugLag(f,c,x0,y0,"bfgs")
    println("  xk=",xk,", f(xk)=", fxk," and its=",k)
end

Quasi-Newton method (BFGS):
For problem 6: 
  xk=[1.000000000033154,1.0000000000657836], f(xk)=1.0991826799507169e-21 and its=2
For problem 7: 
  xk=[-2.0747818972942065e-39,1.732050809053286], f(xk)=-1.732050809053286 and its=7
For problem 8: 
  xk=[4.601594917913624,1.9558436060828601], f(xk)=1 and its=2
For problem 9: 
  xk=[-2.999999999999783,-4.000000000000051], f(xk)=-0.4999999999999665 and its=3


From the results above: for problem 6-9,  both Newton's and quasi-Newton's methods converge in less than 10 steps ($|c(x)|<1e-8$ and $\|g-J'y\|<1e-8$).